In [ ]:
from google.colab import drive

folder = 'project_VeP'#@param{type:"string"}

drive.mount('/content/gdrive')
print('Connecting to Drive....')

%cd /content/gdrive/MyDrive/$folder/
print('Done!')

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt
import torch
print("Repo Clone Completed!")

In [ ]:
epoch = 300 #@param {type:"slider", min:100, max:1000, step:10}
training_option = 'Train new CNN' #@param ["Train new CNN", "Resume an interrupted Training"]
weight = 'yolov5m.pt' #@param["yolov5s.pt","yolov5m.pt","yolov5l.pt","yolov5x.pt"]
batch = 64 #@param {type:"slider", min:1, max:64, step:1}

if training_option == 'Train new CNN':
  !python /content/gdrive/MyDrive/$folder/yolov5/train.py --img 512 --batch $batch --epochs $epoch --data /content/gdrive/MyDrive/$folder/training_dataset/data.yaml --weights $weight --cache --project /content/gdrive/MyDrive/$folder/training_results/
else:
  !python /content/gdrive/MyDrive/$folder/yolov5/train.py --resume /content/gdrive/MyDrive/$folder/training_results/export/weights/last.pt

#Metriche
Il codice seguente calcola le metriche su delle immagini di test. Vengono poi confrontati i grafici della curva per la F1, sia nel caso del training che sulle immagini di test.

In [ ]:
!python /content/gdrive/MyDrive/$folder/yolov5/val.py --data /content/gdrive/MyDrive/$folder/test_dataset/data.yaml --weights /content/gdrive/MyDrive/$folder/training_results/exp5/weights/best.pt --project /content/gdrive/MyDrive/$folder/test_results/ --imgsz=512

In [ ]:
#Printing result to confront
from PIL import Image

precision_training = '/content/gdrive/MyDrive/'+folder+'/training_results/exp5/F1_curve.png'
precision_test = '/content/gdrive/MyDrive/'+folder+'/test_results/exp2/F1_curve.png'

im1 = Image.open(precision_training)
im2 = Image.open(precision_test)

print("F1 Training")
display(im1)
print("F1 Test")
display(im2)

#Detection
Salviamo le soltanto le label alla fine della detection, in questo modo disegniamo noi le bounding box.

In [ ]:
!python /content/gdrive/MyDrive/$folder/yolov5/detect.py --source /content/gdrive/MyDrive/$folder/detection/exp --weights /content/gdrive/MyDrive/$folder/training_results/exp2/weights/best.pt --project /content/gdrive/MyDrive/$folder/detection_results/ --imgsz 512 --save-txt --save-conf --exist-ok --nosave

In [ ]:
import glob
import os
import shutil
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

path = '/content/gdrive/MyDrive/'+folder+'/detection_results/exp/labels/'
img_path = '/content/gdrive/MyDrive/'+folder+'/detection/exp/'
save_path = '/content/gdrive/MyDrive/'+folder+'/detection_results/exp/'

#shutil.copytree(img_path, save_path)

for filename in glob.glob(os.path.join(path, '*.txt')):
  with open(os.path.join(os.getcwd(),filename), 'r') as f:
    bounding_boxes = None
    confidences = None
    image_name = os.path.basename(filename).replace('txt', 'jpg')
    for line in f:
      cl, label_x, label_y, label_w, label_h, conf = line.split(' ')
      b = float(conf)
      a = np.array([float(label_x),float(label_y),float(label_w),float(label_h)])
      bounding_boxes = (np.vstack((bounding_boxes, a)) if (bounding_boxes is not None) else a)
      confidences = (np.vstack((confidences, b)) if (confidences is not None) else b)

    #Image Elaboration
    image = cv.imread(img_path+image_name)
    [image_height, image_width, levels] = image.shape
    
    conf_max = np.amax(confidences)

    [x,y] = [image_height,image_width]
    [w,h] = [0,0]
    for i in range(0,bounding_boxes.shape[0]):
      if(bounding_boxes.ndim == 1):
        center_x_1 = int(bounding_boxes[0]*image_width)
        center_y_1 = int(bounding_boxes[1]*image_height) 
        width_1 = int(bounding_boxes[2]*image_width) 
        height_1 = int(bounding_boxes[3]*image_height)
        x = int(center_x_1 - (width_1/2))
        y = int(center_y_1 - (height_1/2))
        w = int(center_x_1 + (width_1/2))
        h = int(center_y_1 + (height_1/2))
      else:
        center_x_1 = int(bounding_boxes[i,0]*image_width)
        center_y_1 = int(bounding_boxes[i,1]*image_height) 
        width_1 = int(bounding_boxes[i,2]*image_width) 
        height_1 = int(bounding_boxes[i,3]*image_height)
        vert_x = int(center_x_1 - (width_1/2))
        vert_y = int(center_y_1 - (height_1/2))
        vert_w = int(center_x_1 + (width_1/2))
        vert_h = int(center_y_1 + (height_1/2))
        if(vert_x < x):
          x = vert_x
        if(vert_y < y):
          y = vert_y
        if(vert_w > w):
          w = vert_w
        if(vert_h > h):
          h = vert_h
    
    #Create Rect
    cv.rectangle(image, (x,y), (w,h), (0,0,255), 4)
    cv.putText(image, 'crosswalk ' + "%.2f" % conf_max, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    
    #Saving Image
    print('Saving ' + image_name)
    cv.imwrite(save_path+image_name, image)